### Imports

In [31]:
import os
import sys

sys.path.append("../scripts")

from math import radians, sin, cos, sqrt, atan2
import sentinel, evalscripts
import matplotlib.pyplot as plt
import numpy as np
from geopy.distance import distance
from geopy.point import Point
import yaml

from sentinelhub import CRS, BBox, bbox_to_dimensions, SHConfig

%reload_ext autoreload
%autoreload 2
%matplotlib inline

Creditential

In [32]:
# Fetch credentials for modis
with open("../config.yaml") as file:
    credentials = yaml.safe_load(file)
user = credentials["sentinelhub"]["API_USER"]
password = credentials["sentinelhub"]["API_PASSWORD"]

config = SHConfig(sh_client_id=user, sh_client_secret=password)

if not config.sh_client_id or not config.sh_client_secret:
    print(
        "Warning! To use Process API, please provide the credentials (OAuth client ID and client secret)."
    )

### Setup

In [33]:
# Bounding box coordinates (you can use http://bboxfinder.com/ to find the coordinates of your desired bounding box)
# (-124.45976373769807,48.62786844842075,-115.12341739786068,53.51536369187604)
lon_min = -124.45976373769807
lat_min = 48.62786844842075
lon_max = -115.12341739786068
lat_max = 53.51536369187604

# Desired spacing in kilometers between each image
spacing_km = 100

# Desired resolution of each image
resolution = 300

start_date = "2021-06-29"
end_date = "2021-08-30"

sentinel_request_dir = "../data/sentinel/request"
sentinel_tiff_dir = "../data/sentinel/tiff"
sentinel_merge_dir = "../data/sentinel/merge"

### Get points and bbox for the selected area

In [34]:
points, max_points_in_line, num_lines, points_per_line = (
    sentinel.generate_grid_within_box(lat_min, lon_min, lat_max, lon_max, spacing_km)
)
points = [(y, x) for x, y in points]

aoi_bbox_list = []
for point in points:
    aoi_coords_wgs842 = sentinel.boundingBox(point[0], point[1], spacing_km / 2)
    aoi_bbox_list.append(BBox(bbox=aoi_coords_wgs842, crs=CRS.WGS84))

aoi_size = bbox_to_dimensions(aoi_bbox_list[0], resolution=resolution)

## Print informations

In [35]:
print("Number of points per line: ", max_points_in_line)
print("Number of lines: ", num_lines)
print("Total number of points generated: ", len(points))
print("List of number of points per line: ", points_per_line)
print(f"Image shape at {resolution} m resolution: {aoi_size} pixels")

Number of points per line:  8
Number of lines:  6
Total number of points generated:  48
List of number of points per line:  [8, 8, 8, 8, 8, 8]
Image shape at 300 m resolution: (341, 327) pixels


## Get data

In [36]:
# You can chose your evalscript here between evalscript_ndvi, evalscript_ndvi2 and evalscript_CloudFree_Composite:
evalscript = evalscripts.evalscript_ndvi

ndvi_img_list = []
for aoi_bbox in aoi_bbox_list:
    ndvi_img = sentinel.get_ndvi_img(
        aoi_bbox,
        aoi_size,
        sentinel_request_dir,
        start_date,
        end_date,
        evalscript,
        config,
    )
    ndvi_img_list.append(ndvi_img)

### Concatenation in a tiff file

In [42]:
sentinel.concatenate_tiff_images(
    sentinel_request_dir, sentinel_tiff_dir, sentinel_merge_dir
)

ValueError: No .tiff files found in the directory.

In [38]:
sentinel.create_stitched_image(
    lat_min,
    lon_min,
    lat_max,
    lon_max,
    spacing_km,
    resolution,
    start_date,
    end_date,
    evalscripts.evalscript_ndvi,
    config,
    sentinel_request_dir,
    sentinel_tiff_dir,
    sentinel_merge_dir,
)

['../data/sentinel/request/ada8d995d2dcf0f78c5450530ab0b0c2/response.tiff', '../data/sentinel/request/447b3775bba57a14601383d0dc218b5b/response.tiff', '../data/sentinel/request/077e1500619362684f477fe314be0595/response.tiff', '../data/sentinel/request/0f7777878e89d677345832d076ce7be3/response.tiff', '../data/sentinel/request/e40141db0556210f9d4b375a35176106/response.tiff', '../data/sentinel/request/b75dd3b0344774354f8fc7409f5e14de/response.tiff', '../data/sentinel/request/3e1fbd8aab47c1b51eacce0f230c439e/response.tiff', '../data/sentinel/request/3b7de7e0c84be8573726fffe7f5b1ff4/response.tiff', '../data/sentinel/request/13e788c634ecba9cc050d3569f7956b0/response.tiff', '../data/sentinel/request/384bd15376d4ee99a060f5447fb937ce/response.tiff', '../data/sentinel/request/97b3bf7ec797e96ff87c8e5aa1b6f373/response.tiff', '../data/sentinel/request/af68269495fa0165880eb3a8de00f780/response.tiff', '../data/sentinel/request/e89db5f61c0c624780860f5d6f3edd7c/response.tiff', '../data/sentinel/reques

[ WARN:0@727.299] global loadsave.cpp:241 findDecoder imread_('../data/sentinel/request/ada8d995d2dcf0f78c5450530ab0b0c2/response.tiff'): can't open/read file: check file path/integrity
[ WARN:0@727.299] global loadsave.cpp:241 findDecoder imread_('../data/sentinel/request/447b3775bba57a14601383d0dc218b5b/response.tiff'): can't open/read file: check file path/integrity
[ WARN:0@727.299] global loadsave.cpp:241 findDecoder imread_('../data/sentinel/request/077e1500619362684f477fe314be0595/response.tiff'): can't open/read file: check file path/integrity
[ WARN:0@727.299] global loadsave.cpp:241 findDecoder imread_('../data/sentinel/request/0f7777878e89d677345832d076ce7be3/response.tiff'): can't open/read file: check file path/integrity
[ WARN:0@727.299] global loadsave.cpp:241 findDecoder imread_('../data/sentinel/request/e40141db0556210f9d4b375a35176106/response.tiff'): can't open/read file: check file path/integrity
[ WARN:0@727.299] global loadsave.cpp:241 findDecoder imread_('../data

'../data/sentinel/merge/merged_image_3.tiff'